In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=True)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8419.56it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4653.36it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2575444.80it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8532.47it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4644.58it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 80911.91it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 8664.01it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 4618.89it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 87180.97it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 9002.28it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4609.52it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 66940.29it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
hist_bin_size = 30
model = mz.models.DRMM()
model.params['input_shapes'] = [[10,], [10, hist_bin_size,]]
model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding_input_dim'] = preprocessor.context['vocab_size'] + 1
model.params['embedding_output_dim'] = 300
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DRMM.
Parameter "embedding_trainable" set to True.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 300)      5002200     text_left[0][0]                  
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10, 1)        300         embedding[0][0]                  
__________________________________________________________________________________________________
match_histogram (InputLayer)    (None, 10, 30)       0                                            
__________________________________

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [8]:
# normalize the word embedding for fast histogram generating.
l2_norm = np.sqrt((embedding_matrix*embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [9]:
model.load_embedding_matrix(embedding_matrix)

In [10]:
pred_generator = mz.HistogramDataGenerator(data_pack=predict_pack_processed,
                                           embedding_matrix=embedding_matrix,
                                           hist_bin_size=hist_bin_size, 
                                           hist_mode='LCH')
pred_x, pred_y = pred_generator[:]
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, once_every=1, batch_size=len(pred_y))

In [11]:
train_generator = mz.HistogramPairDataGenerator(train_pack_processed, embedding_matrix, hist_bin_size, 'LCH',
                                                num_dup=2, num_neg=4, batch_size=20)
len(train_generator)

102

In [12]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 8s 77ms/step - loss: 1.5579
Validation: normalized_discounted_cumulative_gain@3(0): 0.45594572901285396 - normalized_discounted_cumulative_gain@5(0): 0.5245861940394168 - mean_average_precision(0): 0.48429476845051783
Epoch 2/30
102/102 [==============================] - 6s 60ms/step - loss: 1.4613
Validation: normalized_discounted_cumulative_gain@3(0): 0.5047168066848209 - normalized_discounted_cumulative_gain@5(0): 0.5738026115744619 - mean_average_precision(0): 0.5280573234874534
Epoch 3/30
102/102 [==============================] - 7s 68ms/step - loss: 1.3457
Validation: normalized_discounted_cumulative_gain@3(0): 0.5691057432493024 - normalized_discounted_cumulative_gain@5(0): 0.6333049583015313 - mean_average_precision(0): 0.5893635062576
Epoch 4/30
102/102 [==============================] - 7s 65ms/step - loss: 1.2436
Validation: normalized_discounted_cumulative_gain@3(0): 0.589530184809194 - normalized_discounted_cumulative_

102/102 [==============================] - 7s 65ms/step - loss: 0.5793
Validation: normalized_discounted_cumulative_gain@3(0): 0.6060529381418986 - normalized_discounted_cumulative_gain@5(0): 0.6532253603499287 - mean_average_precision(0): 0.6167519344641459
Epoch 28/30
102/102 [==============================] - 7s 71ms/step - loss: 0.5755
Validation: normalized_discounted_cumulative_gain@3(0): 0.6068646356166916 - normalized_discounted_cumulative_gain@5(0): 0.651884182315063 - mean_average_precision(0): 0.6169867739551335
Epoch 29/30
102/102 [==============================] - 7s 68ms/step - loss: 0.5712
Validation: normalized_discounted_cumulative_gain@3(0): 0.610531598681791 - normalized_discounted_cumulative_gain@5(0): 0.6571485548065488 - mean_average_precision(0): 0.6230152265275567
Epoch 30/30
102/102 [==============================] - 7s 66ms/step - loss: 0.5672
Validation: normalized_discounted_cumulative_gain@3(0): 0.60199391767509 - normalized_discounted_cumulative_gain@5(0):

In [25]:
import keras
import keras.backend as K
import pandas as pd

In [26]:
for idx,layer in enumerate(model.backend.layers):
    print({idx:layer.name})

{0: 'text_left'}
{1: 'embedding'}
{2: 'dense_1'}
{3: 'match_histogram'}
{4: 'attention_mask'}
{5: 'dense_2'}
{6: 'attention_probs'}
{7: 'dense_3'}
{8: 'dot_1'}
{9: 'flatten_1'}
{10: 'dense_4'}


In [14]:
left = np.array([[2, 3, 4, 5, 0, 0]])
right = np.array([[[0.1, 0.2, 0.3, 0.4]]])

In [22]:
attent_fn = K.function([model.backend.layers[6].input, K.learning_phase()], [model.backend.layers[6].output])

In [23]:
res = attent_fn([left, right])

In [29]:
text_left = "you would like to book a room outside of the dates"
text_right = "you prefer to book your own accommodation"
left = pd.DataFrame([['id0', text_left]])
right = pd.DataFrame([['id1', text_right]])
relation = pd.DataFrame([['id0', 'id1']])
test_sample = mz.DataPack(left=left, right=right, relation=relation)
test_sample

In [31]:
preprocessor.save('./drmm_preprocessor')

In [32]:
new_preprocessor = mz.load_preprocessor('./drmm_preprocessor')

In [35]:
x1 = preprocessor.transform(test_sample)

KeyError: 'text_left'